In [2]:
from astropy.io import fits
import numpy as np
import glob
import os
import pathlib
import re
from datetime import datetime

In [29]:
# hdulist = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050929/st_2418807291_p.fit") #scattered
#hdulist = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051013/st_2456811983_w.fit") #smear
hdulist = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20040517/st_1035463706_p.fit") #moon
# hdulist = fits.open("../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489428266_b.fit") #itokawa surface
hdu = hdulist[0]
data = hdu.data
header = hdu.header
date = header["UTC_0"]
date = re.split('[-T]',date)
l_year = 2003
l_month = 5
l_day = 9
l_date = datetime(l_year,l_month,l_day)
year = int(date[0])
month = int(date[1])
day = int(date[2])
date = datetime(year,month,day)
days = date - l_date
DAY = days.days
xs = header["NAXIS1"]
ys = header["NAXIS2"]
startH = header["START_H"]
startV = header["START_V"]
filt = header["FILTER_0"]
tEXP = header["EXP_0"]
num_bin = header["BINNING"]

In [30]:
#smear correction
def get_smear(NV, NH, tEXP, Isky, data):
    tVCT = 12 * 1024 * 10 ** -6#μs

    result = np.zeros((NV,NH))
    Ismear = np.zeros((NV,NH))

    for H in range (NH):
        Ismear[:,H] = 0
        for V in range (NV):
            Ismear[:,H] += (tVCT / (tVCT + tEXP)) * ((data[V,H] - Isky)/1024)
        result[:,H] =data[:,H] - Ismear[:,H]
        # print(Ismear,data[:,H] , "result",result[:,H] )

    return result, Ismear

# hdu = fits.PrimaryHDU(result)
# hdulist = fits.HDUList([hdu])
# hdulist.writeto('smear_cor.fits',overwrite=True)


In [31]:
#flat correction
def get_flat(filt):
    result = 0
    home_dir = os.path.expanduser("~")
    p = os.path.join(home_dir, "Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/preflight")
    for f in glob.glob(p+"/flat_"+filt+".fit"):
        result = fits.open(f)
        result = result[0].data
    return result
def flat_cor(filt, data,startH,startV,x_axs,y_axs):
    result = 0
    flat_im = get_flat(filt)
    
    result = data/flat_im[startV:startV+y_axs,startH:startH+x_axs]

    return result

In [32]:
#bias correction
def get_bias(DAY):
    result = 0

    B0 = 3.18 * pow(10,2)
    B1 = -4.12 * pow(10,-2)
    B2 = 2.00 * pow(10,-5)

    result = B0 + (B1 * DAY) + (B2 * pow(DAY,2))

    return result

get_bias(DAY)

305.38872000000003

In [7]:
#dark correction
def get_dark(T):
    result = 0
    d0_im = 3.60
    d1_im = 0.123
    d0_st = 2.96
    d1_st = 0.140

    result = d0_im*np.exp(d1_im*T) + d0_st*np.exp(d1_st*T)
    return result

In [33]:
#scattered light correction
def make_coord(x,y):
    a = np.array([[[x,y]]])
    coordinate = np.tile(a,((2*x)-1,(2*y)-1,1))
    for i in range((2*x)-1):
        for j in range((2*y)-1):
            coordinate[i][j]=[i,j]
    return coordinate

def make_base(x,y):
    a = np.array([[[x-1,y-1]]])
    base = np.tile(a,((2*x)-1,(2*y)-1,1))
    return base

def calc_dist(coord,base):
    a = base-coord
    b = np.sum(np.square(a), axis=2)
    c = np.sqrt(b)
    return c

def scattered_light(xs, ys, filt, data):
    result = np.zeros((header["NAXIS1"],header["NAXIS2"]))
    # result = np.zeros((xs,ys))
    Ai = {"ul":{"1":12.0,"2":8.0,"3":1.2,"4":1.0,"5":0.8,"6":0.7},
      "b":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "v":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "w":{"1":10.0,"2":1.5,"3":0.6,"4":0.8,"5":0.7,"6":0.6},
      "x":{"1":9.0,"2":3.5,"3":2.0,"4":2.7,"5":2.2,"6":0.5},
      "p":{"1":10.0,"2":5.0,"3":8.3,"4":4.0,"5":6.4,"6":1.8},
      "zs":{"1":50.0,"2":16.0,"3":6.0,"4":9.0,"5":9.5,"6":4.5}
    }
    filter_u=Ai[filt]
    sigma = [8,16,32,64,110,710]
    coord = make_coord(xs,ys)
    base = make_base(xs,ys)
    r = calc_dist(coord, base)
    fufoc = 0
    for k in range(6):
        num = str(k + 1)
        fufoc = fufoc + ((filter_u[num]*np.power(10.,-4))/(np.sqrt(2*np.pi)*sigma[k]))*np.exp(-np.power(r,2)/(2*np.power(sigma[k],2)))
    for x in range(xs):
        for y in range(ys):
            result[x][y]=np.sum(np.multiply(fufoc[(xs-1)-x:(2*xs-1)-x,(ys-1)-y:(2*ys-1)-y],data))
    return result



In [34]:
#moon
bias = get_bias(DAY)
bias_data = data - bias
smear_data ,Ismear= get_smear(xs, ys, tEXP, get_bias(DAY), bias_data)
flat_data = flat_cor(filt,smear_data,startH,startV,xs,ys)
scattered_data = scattered_light(xs, ys, filt, flat_data)

In [60]:
#read_out
bias = get_bias(DAY)
bias_data = data - bias
smear_data ,Ismear= get_smear(xs, ys, tEXP, get_bias(DAY), bias_data)

hdu = fits.PrimaryHDU(smear_data)
hdulist = fits.HDUList([hdu])
hdulist.writeto('readout_cor.fit',overwrite=True)

In [9]:
flat_data = flat_cor(filt,data,startH,startV,xs,ys)

hdu = fits.PrimaryHDU(flat_data)
hdulist = fits.HDUList([hdu])
hdulist.writeto('my_flat.fit',overwrite=True)

In [57]:
flat_data = flat_cor(filt,data,startH,startV,xs,ys)
result = scattered_light(xs, ys, filt, flat_data)
data1 = flat_data-result

hdu = fits.PrimaryHDU(data1)
hdulist = fits.HDUList([hdu])
hdulist.writeto('itokawa_cor.fit',overwrite=True)

In [26]:
result = flat_data - scattered_data

hdu = fits.PrimaryHDU(result)
hdulist = fits.HDUList([hdu])
hdulist.writeto('moon_cor.fit',overwrite=True)

In [22]:
#test
flat_data = flat_cor(filt,data,startH,startV,xs,ys)
result = scattered_light(xs, ys, filt, flat_data)
data1 = flat_data-result

hdu = fits.PrimaryHDU(data1)
hdulist = fits.HDUList([hdu])
hdulist.writeto('test_cor.fit',overwrite=True)

In [20]:
print((result==result1).all())

True


In [27]:
filtered_values = [value for row in result for value in row if value >= 500]

# 条件を満たす要素の平均を計算
if filtered_values:
    average = sum(filtered_values) / len(filtered_values)
    print("500以上の要素の平均:", average)
else:
    print("500以上の要素が見つかりませんでした。")

500以上の要素の平均: 754.3723029871288
